In [9]:
# Import libraries and set desired options
%matplotlib inline
from matplotlib import pyplot as plt

import pickle
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

Notebook by Yuri Kashnitsky, edited by Ivan Komarov. 

In this competition we are going to analyze a sequence of websites visited by a person to predict whether this person is Alice or not. The metric of evaluation is [ROC AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). 

###  Data Downloading and Transformation
First, read the training and test sets. 

In [23]:
# создаем названия колонок с датами, чтобы их потом правильно спарсить
times = ['time'+str(i) for i in range(1,11)]
times

['time1',
 'time2',
 'time3',
 'time4',
 'time5',
 'time6',
 'time7',
 'time8',
 'time9',
 'time10']

In [24]:
# Read the training and test data sets and parse dates
train_df = pd.read_csv('train.csv',
                       index_col='session_id', parse_dates=times)

test_df = pd.read_csv('test.csv',
                      index_col='session_id', parse_dates=['time1'])

# Sort the data by time1
train_df = train_df.sort_values(by='time1')

In [25]:
# Look at the first rows of the training set
# train_df.head()
train_df.sample(10).sort_values(by='time1')

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
87945,28288,2013-12-04 09:49:00,270.0,2013-12-04 09:49:01,914.0,2013-12-04 09:49:04,913.0,2013-12-04 09:49:04,912.0,2013-12-04 09:49:04,...,2013-12-04 09:49:04,167.0,2013-12-04 09:49:11,167.0,2013-12-04 09:49:12,1295.0,2013-12-04 09:49:12,1296.0,2013-12-04 09:49:12,0
13253,1035,2014-01-27 14:18:15,23.0,2014-01-27 14:18:15,21.0,2014-01-27 14:18:15,21.0,2014-01-27 14:18:31,23.0,2014-01-27 14:18:34,...,2014-01-27 14:18:35,22.0,2014-01-27 14:18:55,23.0,2014-01-27 14:18:55,21.0,2014-01-27 14:18:55,305.0,2014-01-27 14:27:19,0
43516,7608,2014-01-28 13:17:22,7609.0,2014-01-28 13:17:22,15762.0,2014-01-28 13:17:22,7610.0,2014-01-28 13:17:23,7608.0,2014-01-28 13:17:23,...,2014-01-28 13:17:23,15762.0,2014-01-28 13:17:23,3426.0,2014-01-28 13:17:23,3425.0,2014-01-28 13:17:23,15758.0,2014-01-28 13:17:23,0
57493,1307,2014-02-19 11:12:14,81.0,2014-02-19 11:12:14,76.0,2014-02-19 11:12:14,1057.0,2014-02-19 11:12:14,77.0,2014-02-19 11:12:14,...,2014-02-19 11:12:15,77.0,2014-02-19 11:12:15,76.0,2014-02-19 11:12:15,81.0,2014-02-19 11:12:15,78.0,2014-02-19 11:12:15,0
196498,23,2014-02-24 13:34:19,52.0,2014-02-24 13:34:19,16058.0,2014-02-24 13:34:19,48.0,2014-02-24 13:34:19,1074.0,2014-02-24 13:34:24,...,2014-02-24 13:34:30,3270.0,2014-02-24 13:34:30,23.0,2014-02-24 13:34:31,3270.0,2014-02-24 13:34:31,3272.0,2014-02-24 13:34:31,0
182058,6246,2014-02-25 09:38:05,6246.0,2014-02-25 09:38:06,10374.0,2014-02-25 09:38:07,6246.0,2014-02-25 09:38:07,6246.0,2014-02-25 09:38:08,...,2014-02-25 09:38:09,6246.0,2014-02-25 09:38:10,6246.0,2014-02-25 09:38:11,6246.0,2014-02-25 09:38:12,6246.0,2014-02-25 09:38:13,0
40236,38267,2014-02-27 11:31:21,38267.0,2014-02-27 11:31:22,38267.0,2014-02-27 11:31:52,38267.0,2014-02-27 11:31:54,38267.0,2014-02-27 11:32:01,...,2014-02-27 11:32:02,38267.0,2014-02-27 11:32:10,38267.0,2014-02-27 11:32:11,38267.0,2014-02-27 11:32:12,38267.0,2014-02-27 11:32:28,0
68771,2949,2014-03-24 13:01:10,5129.0,2014-03-24 13:01:10,2951.0,2014-03-24 13:01:10,5130.0,2014-03-24 13:01:10,5179.0,2014-03-24 13:01:10,...,2014-03-24 13:01:10,5129.0,2014-03-24 13:01:11,2951.0,2014-03-24 13:01:11,1324.0,2014-03-24 13:01:11,35.0,2014-03-24 13:01:12,0
212559,2612,2014-03-25 12:09:29,2736.0,2014-03-25 12:09:29,23.0,2014-03-25 12:09:29,2738.0,2014-03-25 12:09:29,2740.0,2014-03-25 12:09:29,...,2014-03-25 12:09:29,2736.0,2014-03-25 12:09:30,2742.0,2014-03-25 12:09:30,29728.0,2014-03-25 12:09:32,2750.0,2014-03-25 12:09:33,0


In [26]:
test_df.sample(10).sort_values(by='time1')

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
40779,570,2014-01-08 10:32:45,21.0,2014-01-08 10:32:54,616.0,2014-01-08 10:33:37,677.0,2014-01-08 10:33:44,21.0,2014-01-08 10:36:56,616.0,2014-01-08 10:37:33,678.0,2014-01-08 10:39:13,570.0,2014-01-08 10:40:01,677.0,2014-01-08 10:41:44,616.0,2014-01-08 10:42:51
69435,362,2014-01-09 14:34:25,364.0,2014-01-09 14:34:26,360.0,2014-01-09 14:34:26,363.0,2014-01-09 14:34:26,167.0,2014-01-09 14:34:26,362.0,2014-01-09 14:34:26,1651.0,2014-01-09 14:34:26,362.0,2014-01-09 14:34:27,360.0,2014-01-09 14:34:27,364.0,2014-01-09 14:34:27
34779,22,2014-02-08 11:22:43,5194.0,2014-02-08 11:22:43,37.0,2014-02-08 11:22:43,14590.0,2014-02-08 11:22:46,261.0,2014-02-08 11:22:46,148.0,2014-02-08 11:22:46,35.0,2014-02-08 11:22:47,461.0,2014-02-08 11:22:47,29.0,2014-02-08 11:22:47,30.0,2014-02-08 11:22:47
59291,306,2014-02-11 11:29:38,305.0,2014-02-11 11:29:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24994,513,2014-02-11 17:00:04,747.0,2014-02-11 17:00:05,23.0,2014-02-11 17:00:05,29.0,2014-02-11 17:00:06,747.0,2014-02-11 17:00:06,148.0,2014-02-11 17:00:06,752.0,2014-02-11 17:00:14,148.0,2014-02-11 17:00:26,21.0,2014-02-11 17:00:32,46766.0,2014-02-11 17:00:33
23368,30,2014-02-12 11:24:41,6552.0,2014-02-12 11:24:41,32.0,2014-02-12 11:24:41,5657.0,2014-02-12 11:24:41,6554.0,2014-02-12 11:24:42,6552.0,2014-02-12 11:24:42,6554.0,2014-02-12 11:24:43,35.0,2014-02-12 11:24:44,33.0,2014-02-12 11:24:44,6554.0,2014-02-12 11:24:44
25624,3690,2014-03-05 09:11:03,3690.0,2014-03-05 09:11:04,29.0,2014-03-05 09:11:06,3690.0,2014-03-05 09:11:06,3690.0,2014-03-05 09:11:07,29.0,2014-03-05 09:11:07,3690.0,2014-03-05 09:11:14,3690.0,2014-03-05 09:11:15,3690.0,2014-03-05 09:11:16,3690.0,2014-03-05 09:11:17
39542,782,2014-04-08 08:29:21,782.0,2014-04-08 08:29:51,782.0,2014-04-08 08:30:21,782.0,2014-04-08 08:30:51,782.0,2014-04-08 08:31:21,780.0,2014-04-08 08:31:42,782.0,2014-04-08 08:31:52,780.0,2014-04-08 08:32:13,782.0,2014-04-08 08:32:22,780.0,2014-04-08 08:32:24
27813,896,2014-04-09 14:23:52,2815.0,2014-04-09 14:23:52,23.0,2014-04-09 14:23:52,335.0,2014-04-09 14:23:53,2296.0,2014-04-09 14:23:53,2815.0,2014-04-09 14:23:53,335.0,2014-04-09 14:23:58,5307.0,2014-04-09 14:23:59,335.0,2014-04-09 14:23:59,335.0,2014-04-09 14:24:00


In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 27554 to 11690
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   site1   253561 non-null  int64         
 1   time1   253561 non-null  datetime64[ns]
 2   site2   250098 non-null  float64       
 3   time2   250098 non-null  datetime64[ns]
 4   site3   246919 non-null  float64       
 5   time3   246919 non-null  datetime64[ns]
 6   site4   244321 non-null  float64       
 7   time4   244321 non-null  datetime64[ns]
 8   site5   241829 non-null  float64       
 9   time5   241829 non-null  datetime64[ns]
 10  site6   239495 non-null  float64       
 11  time6   239495 non-null  datetime64[ns]
 12  site7   237297 non-null  float64       
 13  time7   237297 non-null  datetime64[ns]
 14  site8   235224 non-null  float64       
 15  time8   235224 non-null  datetime64[ns]
 16  site9   233084 non-null  float64       
 17  time9   233084 non-null  d

In [15]:
##
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   site1   82797 non-null  int64         
 1   time1   82797 non-null  datetime64[ns]
 2   site2   81308 non-null  float64       
 3   time2   81308 non-null  object        
 4   site3   80075 non-null  float64       
 5   time3   80075 non-null  object        
 6   site4   79182 non-null  float64       
 7   time4   79182 non-null  object        
 8   site5   78341 non-null  float64       
 9   time5   78341 non-null  object        
 10  site6   77566 non-null  float64       
 11  time6   77566 non-null  object        
 12  site7   76840 non-null  float64       
 13  time7   76840 non-null  object        
 14  site8   76151 non-null  float64       
 15  time8   76151 non-null  object        
 16  site9   75484 non-null  float64       
 17  time9   75484 non-null  object        
 18  site10

The training data set contains the following features:

- **site1** – ID of the first visited website in the session
- **time1** – visiting time for the first website in the session
- ...
- **site10** – ID of the tenth visited website in the session
- **time10** – visiting time for the tenth website in the session
- **target** – target variable, equals 1 for Alice's sessions, and 0 otherwise
    
**User sessions end either if a user has visited ten websites or if a session has lasted over thirty minutes.**

There are some empty values in the table, it means that some sessions contain less than ten websites. Replace empty values with 0 and change columns types to integer. Also load the websites dictionary and check how it looks:

In [27]:
# Change site1, ..., site10 columns type to integer and fill NA-values with zeros
sites = ['site'+str(i) for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# Load websites dictionary
with open(r"site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)
    
# r before a string means "raw", i.e. take the string as it comes,
# e.g. as a file path without interpreting special symbols like \n

print('Websites total:', len(site_dict))



Websites total: 48371


In [28]:
# See what's in the dict
list(site_dict.items())[:5]

[('www.abmecatronique.com', 25075),
 ('groups.live.com', 13997),
 ('majeureliguefootball.wordpress.com', 42436),
 ('cdt46.media.tourinsoft.eu', 30911),
 ('www.hdwallpapers.eu', 8104)]

In [29]:
# Size of the sets
print(test_df.shape, train_df.shape)

(82797, 20) (253561, 21)


In [30]:
# What's inside the train
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
27554,41475,2013-11-15 07:39:35,6725,2013-11-15 07:39:35,6725,2013-11-15 07:39:36,41475,2013-11-15 07:39:36,41476,2013-11-15 07:39:40,...,2013-11-15 07:39:41,6725,2013-11-15 07:42:50,41475,2013-11-15 07:42:50,41476,2013-11-15 07:42:50,6725,2013-11-15 07:44:25,0
81350,41476,2013-11-15 07:44:25,41475,2013-11-15 07:44:25,41476,2013-11-15 07:57:45,6725,2013-11-15 07:57:45,41475,2013-11-15 07:57:45,...,2013-11-15 07:57:46,41476,2013-11-15 07:57:47,6725,2013-11-15 07:57:49,41475,2013-11-15 07:57:49,41476,2013-11-15 07:57:49,0
234665,4802,2013-11-15 07:52:17,23,2013-11-15 07:52:18,4803,2013-11-15 07:52:19,38,2013-11-15 07:52:19,38,2013-11-15 07:52:20,...,2013-11-15 07:52:20,4804,2013-11-15 07:52:23,21,2013-11-15 07:52:26,23,2013-11-15 07:52:26,22,2013-11-15 07:52:28,0
97610,23,2013-11-15 07:52:28,23,2013-11-15 07:52:29,22,2013-11-15 07:52:37,21,2013-11-15 07:52:37,63,2013-11-15 07:55:10,...,2013-11-15 07:55:10,784,2013-11-15 07:55:56,4804,2013-11-15 07:57:50,4804,2013-11-15 08:01:18,784,2013-11-15 08:01:26,0
161358,41476,2013-11-15 07:57:50,41476,2013-11-15 07:57:51,6725,2013-11-15 07:59:34,41475,2013-11-15 07:59:34,41476,2013-11-15 07:59:34,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0


For the very basic model, we will use only the visited websites in the session (we will not take into account timestamp features). 

*Alice has her favorite sites, and the more often you see these sites in the session, the higher probability that this is an Alice session, and vice versa.*

Let us prepare the data, we will take only features `site1, site2, ... , site10` from the whole dataframe. Keep in mind that the missing values are replaced with zero. Here is what the first rows of the dataframe look like:

In [31]:
train_df[sites].head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
27554,41475,6725,6725,41475,41476,41476,6725,41475,41476,6725
81350,41476,41475,41476,6725,41475,41476,41476,6725,41475,41476
234665,4802,23,4803,38,38,4804,4804,21,23,22
97610,23,23,22,21,63,66,784,4804,4804,784
161358,41476,41476,6725,41475,41476,0,0,0,0,0


In [ ]:
## запомню train_df и test_df

In [60]:
train_df.to_csv('train_df.csv')
test_df.to_csv('test_df.csv')

Since IDs of sites have no meaning (does not matter if a site has an ID of 1 or 100), we need to think about how to encode the meaning of "this site in a session means higher probablity that it is an Alice session". 

We will use a technique called ["bag of words plus n-gram model"](https://en.wikipedia.org/wiki/Bag-of-words_model).

We will make a "site-session" matrix analogous to the term-document matrix.

We are not the first, and luckily there is a function CountVectorizer that will implement the above model. Type help(CountVectorizer) to learn about the function. 

We will now initialize a "cv" (CountVectorizer's) instance which we need to train. 

We will use the following parameters:

_ngram range=(1, 3)_ - here we decide that we will use 
1) the name of the site, 
2) two consecutive site names, and 
3) three consecutive site names as features. 
E.g. "google.com" or "google.com vk.com" or "google.com vk.com groups.live.com". 

CountVectorizer will create a large dictionary of 1, 2, and 3-gram strings of sites represented by their numerical IDs. However, this dictionary will be so so large that we may run into trouble with memory or we will just be inefficent chasing phantom combinations.

We will thus limit the dictionary to 50K of the most frequent n-grams:

_max features=50000_

Here is our empty instance:

In [ ]:
## Как я понял: создадим матрицу "мешок слов".
## Колонки из 1 назавания сайта, из 2 названий и 3 названий.
## Отберем 50 тысяч самых частых.
## Строки - наши сессии

In [21]:
cv = CountVectorizer(ngram_range=(1, 3), max_features=50000)

CountVectorizer accepts "document strings", so let's prepare a string of our "documents" (i.e. sites), divided by space. Since the string will be huge, we will write this string in a text file using pandas:

In [22]:
train_df[sites].fillna(0).to_csv('train_sessions_text.txt', 
                                 sep=' ', index=None, header=None)
test_df[sites].fillna(0).to_csv('test_sessions_text.txt', 
                                sep=' ', index=None, header=None)

Before we start using CountVectorizer, let's see how it works on a sub-set of 5 sessions:

In [23]:
five_sess = pd.read_csv('train_sessions_text.txt', sep=' ', nrows=5, header=None)

In [24]:
five_sess

,0,1,2,3,4,5,6,7,8,9
0,41475,6725,6725,41475,41476,41476,6725,41475,41476,6725
1,41476,41475,41476,6725,41475,41476,41476,6725,41475,41476
2,4802,23,4803,38,38,4804,4804,21,23,22
3,23,23,22,21,63,66,784,4804,4804,784
4,41476,41476,6725,41475,41476,0,0,0,0,0


First of all, let's make an inverse dictionary which gives us a site name for ID.
The direct dictionary came to us like this:

In [27]:
list(site_dict.items())[:3]

[('www.abmecatronique.com', 25075),
 ('groups.live.com', 13997),
 ('majeureliguefootball.wordpress.com', 42436)]

In [28]:
# The inverse dictionary:

new_dict = {}
for key in site_dict:
    new_dict[site_dict[key]] = key

In [29]:
# Let's check what's in it:

list(new_dict.items())[:3]

[(25075, 'www.abmecatronique.com'),
 (13997, 'groups.live.com'),
 (42436, 'majeureliguefootball.wordpress.com')]

In [30]:
# Let's see site names in the five first sessions:

list_sites = []
for row in five_sess.values:
    row_sites = ' '.join([str(i) for i in row if i!=0])
    print(row_sites)
    list_sites.append(row_sites) 

print()
    
list_sites_names = []
for row in five_sess.values:
    row_sites = ' '.join([new_dict[i] for i in row if i!=0])
    print(row_sites)
    list_sites_names.append(row_sites)

41475 6725 6725 41475 41476 41476 6725 41475 41476 6725
41476 41475 41476 6725 41475 41476 41476 6725 41475 41476
4802 23 4803 38 38 4804 4804 21 23 22
23 23 22 21 63 66 784 4804 4804 784
41476 41476 6725 41475 41476

security.debian.org www-fourier.ujf-grenoble.fr www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org backports.debian.org www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org www-fourier.ujf-grenoble.fr
backports.debian.org security.debian.org backports.debian.org www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org backports.debian.org www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org
cnfg.toolbarservices.com www.google.com utils.delta-search.com ajax.googleapis.com ajax.googleapis.com img.babylon.com img.babylon.com www.google.fr www.google.com apis.google.com
www.google.com www.google.com apis.google.com www.google.fr ieonline.microsoft.com go.microsoft.com javadl-esd-secure.oracle.com img.babylon.com i

Here is what the fit and transform method -- i.e. learn the dictionary and make the matrix -- produces in our "cv":
a sparse matrix. Why sparse? Because nrows * dict_size = usually will not fit in memory 
(obviously, our 5 sessions will fit in memory so that we can look at them)

In [31]:
see_vect = cv.fit_transform(list_sites)

# Matrix dimensions: 5 sessions of 60 elements
see_vect

<5x60 sparse matrix of type '<class 'numpy.int64'>'
	with 88 stored elements in Compressed Sparse Row format>

In [32]:
# Here is the dictionary of sites, 1 to 3-gram words. First 6 elements in the matrix:

cv.get_feature_names()[:6]

['21', '21 23', '21 23 22', '21 63', '21 63 66', '22']

In [33]:
# A version with the site names. Note that security.debian.org has ID of 21.

for i, string in enumerate(cv.get_feature_names()):
    if i < 21:
        print (i+1, end=" ")
        for num in string.split():
            print(new_dict[int(num)], end=" ")
        print()

1 www.google.fr 
2 www.google.fr www.google.com 
3 www.google.fr www.google.com apis.google.com 
4 www.google.fr ieonline.microsoft.com 
5 www.google.fr ieonline.microsoft.com go.microsoft.com 
6 apis.google.com 
7 apis.google.com www.google.fr 
8 apis.google.com www.google.fr ieonline.microsoft.com 
9 www.google.com 
10 www.google.com apis.google.com 
11 www.google.com apis.google.com www.google.fr 
12 www.google.com www.google.com 
13 www.google.com www.google.com apis.google.com 
14 www.google.com utils.delta-search.com 
15 www.google.com utils.delta-search.com ajax.googleapis.com 
16 ajax.googleapis.com 
17 ajax.googleapis.com ajax.googleapis.com 
18 ajax.googleapis.com ajax.googleapis.com img.babylon.com 
19 ajax.googleapis.com img.babylon.com 
20 ajax.googleapis.com img.babylon.com img.babylon.com 
21 security.debian.org 


In [34]:
# Here is the session-site matrix, toarrray() helps us to see a sparse matrix since it is not large.

see_vect.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2,
        1, 1, 1, 1, 3, 0, 0, 1, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 2, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3,
        1, 1, 0, 0, 5, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 3, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 

In [35]:
# The first session (row in the matrix) is this:

list_sites_names[0]

'security.debian.org www-fourier.ujf-grenoble.fr www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org backports.debian.org www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org www-fourier.ujf-grenoble.fr'

Let's see how the first site of the first session, "security.debian.org", is recorded in the session-site matrix. 
Its ID is 21 which corresponds to 3. It is the number of times this site was seen in the first session.
Indeed, count for yourself in the cell above. 

In [36]:

first_row = see_vect.toarray()[0]

for one, two in zip(range(60),first_row):
    if one < 21:
        print (one+1, two)


1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 3


Let's go back to all sessions.

Fit `CountVectorizer` to train data and transform the train and test data with it.

In [37]:
%%time

with open('train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)

print(X_train.shape, X_test.shape)

# Note very big dimensions of matrices: 253561 * 50000 = 12678050000 elements in train! Only sparse matrices can take it.

(253561, 50000) (82797, 50000)
Wall time: 20.7 s


In [ ]:
## сохраню целевую переменную для тренировочного датасета

In [47]:
y_train = train_df['target'].values
np.save('y_train', y_train)

In [277]:
sites = ['site'+str(i) for i in range(1, 11)]
times = ['time'+str(i) for i in range(1,11)]

_____________
_____________
______________

# ДОМАШНЕЕ ЗАДАНИЕ

1) Какой процент сессий Элис из всех сессий?

In [22]:
y_train[y_train == 1].size / y_train.size * 100

0.905896411514389

2) В какой день недели Элис появилась в первый раз?

In [98]:
train_df.query("target == 1").sort_values('time1').iloc[0].time1.day_name()

'Friday'

3) Какие года присутствуют в обучающей выборке?

In [102]:
train_df.time1.dt.year.unique()

array([2013, 2014], dtype=int64)

4) Какой сайт в обучающей выборке чаще всего посещает первым Элис и обычный пользователь?

In [117]:
with open(r"site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

In [127]:
new_dict = {val: key for key, val in site_dict.items()}
new_dict

{25075: 'www.abmecatronique.com',
 13997: 'groups.live.com',
 42436: 'majeureliguefootball.wordpress.com',
 30911: 'cdt46.media.tourinsoft.eu',
 8104: 'www.hdwallpapers.eu',
 37491: 'img378.imageshack.us',
 5462: 'ecologie.nature.free.fr',
 35425: 'www.ibcn.intec.ugent.be',
 30151: 'kissanime.com',
 38268: 'www.carolineconduiteformation.com',
 43641: 'images.mystockphoto.com',
 36959: 'journalph.csphares.qc.ca',
 40935: 'www.uqo.ca',
 12346: 'd8d94e0wul1nb.cloudfront.net',
 31023: 'openapi.elong.com',
 31114: 'flamenco-o.blogspot.com',
 28165: 'www.pages-annuaire.net',
 31877: 'smart2000.pagesperso-orange.fr',
 31070: 'fast.forbes.com',
 12938: 'i1-js-14-3-01-10077-536503633-i.init.cedexis-radar.net',
 42002: 'i1-js-14-3-01-11074-716595896-i.init.cedexis-radar.net',
 5671: 'www.pacajob.com',
 32074: 'mathaa.epfl.ch',
 7104: 'cbv.sfr.bench.cedexis.com',
 1939: 'fbcdn-sphotos-b-a.akamaihd.net',
 6708: 'www.mystere-tv.com',
 43589: 'www.mon-ip.fr',
 45241: 'www.aqua-passion.com',
 16195: 

In [128]:
new_dict[train_df[train_df.target == 1].site1.value_counts().idxmax()]

's.youtube.com'

In [129]:
new_dict[train_df[train_df.target == 0].site1.value_counts().idxmax()]

'www.google.fr'

5) Какой сайт в обучающей выборке является вторым по популярности в 2014 году и сколько раз его посещали в 2014 году?

In [192]:
# не придумал лучше, чем перевести в нумпи, преобразовать массив в "строчку" и обратно в панадас для value_counts
train_df_2014 = train_df[train_df.time1.dt.year == 2014][sites]
arr = train_df_2014.to_numpy().reshape(train_df_2014.size)
train_df_2014 = pd.DataFrame(arr)
train_df_2014.iloc[:, 0].value_counts()

21       86090
0        81887
782      77039
23       56333
780      40610
         ...  
25991        1
30085        1
32132        1
19842        1
1376         1
Name: 0, Length: 33244, dtype: int64

In [195]:
new_dict[782]

'annotathon.org'

6) Сколько всего сессий обучающей выборке содержат в себе меньше чем 10 сайтов?

In [205]:
train_df[train_df.site10 == 0].site10.count()

22509

7) Выберите верные утверждения для обучающей выборки:  
- В среднем Элис проводит меньше времени на первой странице, чем другие пользователи
- Медианная длительность посещения первой страницы в среднем больше, чем у Элис
- Лето является самым малоактивным временем года вообще
- Элис наиболее активна в марте и неактивна летом

7а) В среднем Элис проводит меньше времени на первой странице, чем другие пользователи

In [224]:
(train_df[train_df.time1.notna() & train_df.time2.notna() & train_df.target == 1].time2 
 - train_df[train_df.time1.notna() & train_df.time2.notna() & train_df.target == 1].time1)\
.mean()\
<\
(train_df[train_df.time1.notna() & train_df.time2.notna() & train_df.target == 0].time2 
 - train_df[train_df.time1.notna() & train_df.time2.notna() & train_df.target == 0].time1)\
.mean()

True

7б) Медианная длительность посещения первой страницы в среднем больше, чем у Элис

In [228]:
(train_df[train_df.time1.notna() & train_df.time2.notna() & train_df.target == 1].time2 
 - train_df[train_df.time1.notna() & train_df.time2.notna() & train_df.target == 1].time1)\
.median()\
<\
(train_df[train_df.time1.notna() & train_df.time2.notna()].time2 
 - train_df[train_df.time1.notna() & train_df.time2.notna()].time1)\
.median()

False

7в) Лето является самым малоактивным временем года вообще

In [241]:
train_df.time1.dt.month.value_counts()

3     55275
2     53108
12    43557
4     42358
11    31118
1     28058
5        87
Name: time1, dtype: int64

In [ ]:
# с июня по октябрь нет ничего?

In [271]:
train_df.time1.min(), train_df.time1.max()

(Timestamp('2013-11-15 07:39:35'), Timestamp('2014-05-04 17:30:57'))

In [ ]:
# нет. Просто мы копили данные с ноября 2013 до мая 2014

7г) Элис наиболее активна в марте и неактивна летом

In [273]:
train_df.query("target == 1").time1.dt.month.value_counts()

12    610
2     449
11    446
3     399
4     305
1      88
Name: time1, dtype: int64

8) Чему равное медианное значение количества уникальных сайтов в рамках одной сессии обучающей выборки?

In [284]:
train_df[sites].nunique(axis=1).median()

6.0

In [295]:
# если < 10 сайтов, то мы нолики поставили. Чтобы это не влияло, поменял обратно на NaNы.
train_df[sites].replace(0, np.nan).nunique(axis=1, dropna=True).median()

6.0

9) Чему равное медианное значение времени сессии в обучающей выборке (время посещения последнего сайта примем равным нулю)?

In [302]:
(train_df[times].max(axis=1) - train_df[times].replace(0, np.nan).min(axis=1)).median()

Timedelta('0 days 00:00:27')

10) В каком месяце доля сессий Элис среди всех наибольшая?

In [309]:
train_df.query("target==1").time1.groupby(train_df.time1.dt.month).count()\
/ train_df.query("target==0").time1.groupby(train_df.time1.dt.month).count()

time1
1     0.003146
2     0.008527
3     0.007271
4     0.007253
5          NaN
11    0.014541
12    0.014204
Name: time1, dtype: float64

_____________
_____________
______________

In [ ]:
## повтор

In [1]:
# Import libraries and set desired options
%matplotlib inline
from matplotlib import pyplot as plt

import pickle
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [2]:
y_train = np.load('y_train.npy')

In [3]:
cv = CountVectorizer(ngram_range=(1, 3), max_features=50000)

In [4]:
%%time

with open('train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)

print(X_train.shape, X_test.shape)

# Note very big dimensions of matrices: 253561 * 50000 = 12678050000 elements in train! Only sparse matrices can take it.

(253561, 50000) (82797, 50000)
Wall time: 25.7 s


In [83]:
##
train_df = pd.read_csv('train_df.csv', index_col=0, parse_dates=list(range(2, 21,2)))
test_df = pd.read_csv('test_df.csv', index_col=0, parse_dates=['time1'])

### Training the first model

So, we have an algorithm and data for it. Let us build our first model, using [logistic regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) implementation from ` Sklearn` with default parameters. We will use the first 90% of the data for training (the training data set is sorted by time) and the remaining 10% for validation. Let's write a simple function that returns the quality of the model and then train our first classifier:

In [5]:
def get_auc_lr_valid(X, y, C=1.0, seed=17, ratio = 0.9):
    
    # Split the data into the training and validation sets
    idx = int(round(X.shape[0] * ratio))
    
    # Classifier training
    lr = LogisticRegression(C=C, random_state=seed, solver='lbfgs', max_iter=500).fit(X[:idx, :], y[:idx])
    
    # Prediction for validation set
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    
    # Calculate the quality
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

In [10]:
# Our target variable
# y_train = train_df['target'].values

In [6]:
%%time
# Calculate metric on the validation set. 90% of train data for training. 10% for validation.

print(get_auc_lr_valid(X_train, y_train))

0.9122547208092401
Wall time: 32.1 s


In [7]:
%%time
# 50% of train data for training:

get_auc_lr_valid(X_train, y_train, ratio=0.5)

Wall time: 16.3 s


0.8225241128295555

In [30]:
# Wow! Big data rules in this task: .82 -> .91

In [17]:
# Function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = range(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [18]:
%%time
# Train the model on the whole training data set
# Use random_state=17 for reproducibility
# Parameter C=1 by default, but here we set it explicitly

lr = LogisticRegression(C=1.0, random_state=17, solver='lbfgs', max_iter=500).fit(X_train, y_train)

# Make a prediction for test data set
y_test = lr.predict_proba(X_test)[:, 1]

# Write it to the file which could be submitted
write_to_submission_file(y_test, 'baseline_1.csv')

The first model demonstrated the quality of 0.9235 on the validation set. Let's take it as the first baseline and starting point. 

In [19]:
##
y_test

array([1.44760032e-03, 1.96265556e-08, 4.84999381e-08, ...,
       2.85689130e-03, 5.66988658e-04, 1.40829491e-05])